In [1]:
import torch
from torch.utils.tensorboard import SummaryWriter
import torch.optim as optim
TB_writer = SummaryWriter()
from GAN_model.resnetGAN import *
from GAN_model.ProteoGAN import *
from utils.dataloader import *
from utils.torch_utils import *
from utils.metrics import *

In [2]:
args_run_name = 'ProteoGAN_GFP_mutational'
# args; TODO: move these to parser for script
args_batch_size=64; args_seed=1; args_eval_n_seq = 500
args_epoch = 300
# GAN specific param
args_z_dim = 128; args_dim = 256
# optimizer specific param
args_lr_Gen = 1e-4; args_lr_Disc = 8e-5; args_loss = 'hinge'
# train discriminator more initially
# according to https://livebook.manning.com/book/gans-in-action/chapter-5/185
args_disc_iters_init = 2; args_disc_iters = 1; args_disc_iters_init_epoch = 50
# make dir
os.makedirs('checkpoints/'+args_run_name, exist_ok=True)
os.makedirs('out/'+args_run_name+'/plot_out/', exist_ok=True)
os.makedirs('out/'+args_run_name+'/seq_out/', exist_ok=True)

In [3]:
seed_everything(args_seed)
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
# keep dataset in cpu
MSA_data = MSA_dataset('GFP_data/gfp_mutational.fasta', 
    torch.device('cpu'), pseudo_MSA=True) 


### Creating data sets

In [4]:
# creating data set with train/val split
train_len = int(MSA_data.length - MSA_data.length*0.15 )
val_len = MSA_data.length - train_len
train_data, val_data = torch.utils.data.random_split(
    MSA_data, [train_len, val_len], 
    generator=torch.Generator().manual_seed(args_seed))
# decoding train and validation sequences for evaluation
val_seqs = [decode_one_seq(torch.argmax(data, dim=-1).detach().cpu()) 
            for data in val_data]
tr_seqs = [decode_one_seq(torch.argmax(data, dim=-1).detach().cpu()) 
           for data in train_data]
# creating dataloader for training data
loader = torch.utils.data.DataLoader(
    train_data, batch_size=args_batch_size, shuffle=True,
    num_workers=1, pin_memory=True)
# fixed parameters
n_chars = 20; seq_len = len(val_seqs[0])

### Evaluating positive and negative control sequences to obtain baseline metrics
Positive: sample of real sequences and simulates a perfect model

Negative: a sample that simulates the worst possible model for each metric: \
    (constant sequence for MMD, repeated sequences for diversity measures).


In [5]:
pos_ctrl_seqs = np.random.choice(tr_seqs, size=len(val_seqs), replace=False)
# constant sequences for MMD
neg_ctrl_seqs_MMD = []
for _ in range(len(val_seqs)):
    rand_aa_idx = np.random.randint(0, high=19)
    rand_seq = decode_one_seq(
        np.random.randint(rand_aa_idx, high=rand_aa_idx+2, size=seq_len))
    neg_ctrl_seqs_MMD.append(rand_seq)
# repeated sequences for diversity measures
rand_seq = decode_one_seq(np.random.randint(0, high=21, size=seq_len))
neg_ctrl_seqs_diversity = [rand_seq for _ in range(len(val_seqs))]
# report baseline
MMD_pos = mmd(seq1=pos_ctrl_seqs, seq2=val_seqs)
MMD_neg = mmd(seq1=neg_ctrl_seqs_MMD, seq2=val_seqs)
Entropy_pos = abs(entropy(seq1=pos_ctrl_seqs, seq2=val_seqs))
Entropy_neg = abs(entropy(seq1=neg_ctrl_seqs_diversity, seq2=val_seqs))
Distance_pos = abs(distance(seq1=pos_ctrl_seqs, seq2=val_seqs))
Distance_neg = abs(distance(seq1=neg_ctrl_seqs_diversity, seq2=val_seqs))

In [6]:
## Lower the better
print('>>> MMD | abs(D_Entropy) | abs(D_Distance)' )
print(f'>>> For postivie control:\n'
      f'{MMD_pos:.5} , {Entropy_pos:.5} , {Distance_pos:.5}')
print(f'>>> For negative control:\n'
      f'{MMD_neg:.5} , {Entropy_neg:.5} , {Distance_neg:.5}')

>>> MMD | abs(D_Entropy) | abs(D_Distance)
>>> For postivie control:
0.0052928 , 0.00033032 , 0.00077894
>>> For negative control:
0.99314 , 0.061507 , 0.13495


In [7]:
def save_checkpoints(discriminator, generator, optim_D, optim_G,  
               scheduler_D, scheduler_G, epoch):
    torch.save({
        'epoch': epoch,
        'disc_state_dict': discriminator.state_dict(),
        'gen_state_dict': generator.state_dict(),
        'optim_D_state_dict': optim_D.state_dict(),
        'optim_G_state_dict': optim_G.state_dict(),
        'scheduler_D_state_dict': scheduler_D.state_dict(),
        'scheduler_G_state_dict': scheduler_G.state_dict(),
    }, os.path.join('checkpoints/'+args_run_name, 'epoch_{}'.format(epoch)))
    return

def load_checkpoints(discriminator, generator, optim_D, optim_G,
                     scheduler_D, scheduler_G, epoch):
    checkpoints = torch.load(
        os.path.join('checkpoints'+args_run_name, 'epoch_{}'.format(epoch)))  
    discriminator.load_state_dict(checkpoints['disc_state_dict'])
    generator.load_state_dict(checkpoints['gen_state_dict'])
    optim_D.load_state_dict(checkpoints['optim_D_state_dict'])
    optim_G.load_state_dict(checkpoints['optim_G_state_dict'])
    scheduler_D.load_state_dict(checkpoints['scheduler_D_state_dict'])
    scheduler_G.load_state_dict(checkpoints['scheduler_G_state_dict'])
    return

## Init model

In [8]:
# init model and optimizer

Disc = Discriminator(args_dim, seq_len=seq_len, n_chars=n_chars).to(device)
Gen = Generator(args_dim, seq_len=seq_len, n_chars=n_chars, 
                z_dim=args_z_dim).to(device)

# Disc = ResNetSN_Discriminator(args_dim, seq_len, n_chars).to(device)
# Gen = ResNetSN_Generator(args_dim, seq_len, n_chars, args_z_dim).to(device)

optim_disc = optim.Adam(filter(lambda p: p.requires_grad, Disc.parameters()),
                        lr=args_lr_Disc, betas=(0.0, 0.9))
optim_gen = optim.Adam(Gen.parameters(), lr=args_lr_Gen, betas=(0.0, 0.9))
# use an exponentially decaying learning rate
scheduler_d = optim.lr_scheduler.ExponentialLR(optim_disc, gamma=0.99)
scheduler_g = optim.lr_scheduler.ExponentialLR(optim_gen, gamma=0.99)

#load_checkpoints(Disc, Gen, optim_disc, optim_gen, scheduler_d, scheduler_g, 0)
#save_checkpoints(Disc, Gen, optim_disc, optim_gen, scheduler_d, scheduler_g, 0)

In [9]:
def sample_generated(generator, epoch, run_name, n_seq=500):
    # random sampling n_seq noise, batched 100 noise at a time
    # return sampled n_seq sequences, also saving them to fasta file
    gen_seqs = []
    max_batch_size = 100
    for _ in range(n_seq//max_batch_size):
        z = torch.randn(max_batch_size, args_z_dim, device=device)
        gen_probs = generator(z)
        batched_seq = [decode_one_seq(
            gen_prob.argmax(-1).cpu().numpy()) for gen_prob in gen_probs]
        gen_seqs += batched_seq
        torch.cuda.empty_cache()
    # saving seqs in fasta
    fasta_name = 'out/' + run_name + '/seq_out/epoch_' + str(epoch) + '.fasta'
    outfile = open(fasta_name, 'w')
    for i, seq in enumerate(gen_seqs):
        seq_id = 'Epoch'+str(epoch)+'_seq'+str(i)
        outfile.write('>' + seq_id + '\n')
        outfile.write(seq + '\n')
    return gen_seqs


def write_eval_TB(writer, MMD, Entropy, Distance, epoch):
    writer.add_scalar(args_run_name+'/MMD',  MMD, epoch)
    writer.add_scalar(args_run_name+'/abs(D_Entropy)', Entropy, epoch)
    writer.add_scalar(args_run_name+'/abs(D_Distance)', Distance, epoch)
    return

def evaluate(epoch):
    # eval mode
    Gen.eval()
    # sample args_eval_n_seq sequences
    gen_seqs = sample_generated(Gen, epoch, args_run_name, n_seq=args_eval_n_seq)
    MMD = mmd(seq1=gen_seqs, seq2=val_seqs)
    Entropy = abs(entropy(seq1=gen_seqs, seq2=val_seqs))
    Distance = abs(distance(seq1=gen_seqs, seq2=val_seqs))
    print(f'#========== evaluating for epoch ' f'{epoch} ' f'==========#')
    print('>>> MMD | abs(D_Entropy) | abs(D_Distance) \n'
          f'{MMD:.5} , {Entropy:.5} , {Distance:.5}')
    # save the gap distribution plot
    #plot_gap_dist(seqs=gen_seqs, run_name=args_run_name, epoch=epoch)
    write_eval_TB(TB_writer, MMD, Entropy, Distance, epoch)
    # revert to train mode
    Gen.train()
    return


In [10]:
def train(epoch):
    for batch_idx, data in enumerate(loader):
        if data.size()[0] != args_batch_size:
            continue
        else:
            data = data.to(device)
        # train discriminator more initially
        if epoch <= args_disc_iters_init_epoch: 
            disc_iters = args_disc_iters_init
        else:
            disc_iters = args_disc_iters
        # update discriminator
        for _ in range(disc_iters):
            z = torch.randn(args_batch_size, args_z_dim, device=device)
            optim_disc.zero_grad()
            optim_gen.zero_grad()
            real_D = Disc(data)
            fake_D = Disc(Gen(z))
            if args_loss == 'hinge':
                disc_loss = nn.ReLU()(1.0 - real_D).mean() + \
                nn.ReLU()(1.0 + fake_D).mean()
            disc_loss.backward()
            optim_disc.step()
        # record output for both fake and real from discriminator
        real_prob_avg = real_D.mean()
        fake_prob_avg = fake_D.mean()
        
        # update generator
        z = torch.randn(args_batch_size, args_z_dim, device=device)
        optim_disc.zero_grad()
        optim_gen.zero_grad()
        if args_loss == 'hinge' or args_loss == 'wasserstein':
            gen_loss = -Disc(Gen(z)).mean()
        gen_loss.backward()
        optim_gen.step()
        
        # write to Tensor board
        iter = len(loader)*epoch + batch_idx
        TB_writer.add_scalars(
            args_run_name+'/GAN loss', {
            'Disc_loss': disc_loss,
            'Gen_loss': gen_loss,
        }, iter)
        TB_writer.add_scalars(
            args_run_name+'/Discriminator out', {
            'Real prob': real_prob_avg,
            'Fake prob': fake_prob_avg,
        }, iter)
        if batch_idx % 60 == 0:
            print('disc loss', disc_loss.item(), 'gen loss', gen_loss.item())
    # update lr scheduler
    scheduler_d.step()
    scheduler_g.step()
    return


In [11]:
for epoch in range(args_epoch):
    train(epoch)
    save_checkpoints(Disc, Gen, optim_disc, optim_gen, 
                     scheduler_d, scheduler_g, epoch)
    if epoch != 0:
        os.remove(os.path.join('checkpoints/'+args_run_name, 
                               'epoch_{}'.format(epoch-1)))
    # eval every 2 epoch
    if epoch % 2 == 0:
        evaluate(epoch)

disc loss 1.9004285335540771 gen loss -0.0528867170214653
disc loss 0.0011554490774869919 gen loss 1.0684306621551514
disc loss 0.0 gen loss 1.135572910308838
disc loss 0.0 gen loss 1.1915593147277832
disc loss 0.0 gen loss 1.2410674095153809
disc loss 0.0 gen loss 1.2312395572662354
disc loss 0.002148861065506935 gen loss 1.2675700187683105
#========== evaluating for epoch 0 ==========#
>>> MMD | abs(D_Entropy) | abs(D_Distance) 
0.59506 , 0.11898 , 1.1474
disc loss 0.011162198148667812 gen loss 1.2037463188171387
disc loss 0.20542268455028534 gen loss 0.8538435101509094
disc loss 0.701797604560852 gen loss 0.3650464117527008
disc loss 1.4825211763381958 gen loss -0.31382477283477783
disc loss 1.824723482131958 gen loss -0.6214114427566528
disc loss 1.9368995428085327 gen loss -0.4585174322128296
disc loss 1.8700339794158936 gen loss -0.6624400615692139
disc loss 1.8650354146957397 gen loss -0.36995619535446167
disc loss 1.8626046180725098 gen loss -0.548788845539093
disc loss 1.84777